# Multilayer Perceptron (MLP) for generating Onion-like News Headlines

Based on Andrej Karpathy's Youtube lecture [Building makemore Part 2: MLP](https://www.youtube.com/watch?v=TCH_1BHY58I) and Bengio et al's paper [A Neural Probabilistic Langue Model](https://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf)

## Imports & Inits

In [ ]:
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True

import pdb, sys, warnings, os, json, torch, re, random, pickle
warnings.filterwarnings(action='ignore')

from IPython.display import display, HTML
from pathlib import Path

import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns

from torch.nn import functional as F
import torch.nn as nn

np.set_printoptions(precision=4)
sns.set_style("darkgrid")
%matplotlib inline

seed = 2468975301
unk_pct = 5
model_dir = '../../models/mlp'

In [ ]:
use_gpu = lambda gpu: 'cuda' if gpu else 'cpu'
device = use_gpu(False)

In [ ]:
onion_df = pd.read_csv('../../data/cleaned_onion_headlines.csv')
texts = onion_df['text'].tolist()
rng = np.random.default_rng(seed)
for i,text in enumerate(texts):  
  if rng.random() <= (unk_pct/100):
    tokens = text.split()
    tokens.insert(rng.integers(0, len(tokens)), '<u>')
    texts[i] = ' '.join(tokens)
  texts[i] = f'<s> {texts[i]} <s>'

In [ ]:
vocab = sorted(set(' '.join(texts).split()))
vocab_size = len(vocab)
stoi = {s:i for i,s in enumerate(vocab)}
itos = {i:s for i,s in enumerate(vocab)}
print(vocab_size)

## Functions

In [ ]:
def build_dataset(texts, stoi, block_size=3, device='cpu'):
  X,Y = [],[]

  for text in texts:
    context = [0] * block_size
    for word in text.split():
      idx = stoi[word]
      X.append(context)
      Y.append(idx)
      context = context[1:] + [idx]
  
  return torch.tensor(X, device=device), torch.tensor(Y, device=device)

In [ ]:
def split_datasets(X, Y, train_pct=0.8, dev_pct=0.1, device='cpu'):
  X = X.to(device)
  Y = Y.to(device)
  r = np.random.RandomState(seed)
  idxs = np.arange(len(X))
  r.shuffle(idxs) 

  n1 = int(train_pct*len(idxs))
  n2 = int((train_pct+dev_pct)*len(idxs))
  

  return X[idxs[:n1]],Y[idxs[:n1]],X[idxs[n1:n2]],Y[idxs[n1:n2]],X[idxs[n2:]],Y[idxs[n2:]]

In [ ]:
def chunks(lst, n):
  """Yield successive n-sized chunks from lst."""
  for i in range(0, len(lst), n):
    yield lst[i:i + n]

def calc_loss(X, Y, params, batch_size):
  C,W1,b1,W2,b2 = params
  n_batches = 0
  loss = 0
  
  for chunk in chunks(range(len(X)), batch_size):
    idxs = list(chunk)
    emb = C[X[idxs]]
    h = torch.tanh(emb.view(-1, input_dim) @ W1 + b1) 
    logits = h @ W2 + b2
    loss += F.cross_entropy(logits, Y[idxs])
    n_batches += 1
  
  return (loss/n_batches).item()

## Build the Dataset

In [ ]:
block_size = 3 # contenxt length
X,Y = [],[]

for text in texts[:1]:
  context = [0] * block_size
  print(text)
  for word in text.split():
    idx = stoi[word]
    X.append(context)
    Y.append(idx)
    print(f"{' '.join(itos[i] for i in context)} -----> {itos[idx]}")
    context = context[1:] + [idx]
X,Y = torch.tensor(X, device=device),torch.tensor(Y, device=device)
# X,Y = build_dataset(texts, stoi)
X.shape, Y.shape

## Building the network

1. Let's build the table look-up `C` from the paper. In the paper, their vocab is 17K words with a 30-dim embedding.
2. Lookup table is the *same* as matrix multiplication with one-hot enc.
3. Pytorch indexing with tensors with example C[X][13,2]



In [ ]:
emb_size = 2
hidden_dim = 100
input_dim = block_size * emb_size

In [ ]:
C = torch.randn((vocab_size, emb_size), device=device)
C.shape

In [ ]:
emb = C[X]
emb.shape

In [ ]:
W1 = torch.randn(input_dim, hidden_dim, device=device)
b1 = torch.randn(hidden_dim, device=device)

Best way to transform tensors `view`

In [ ]:
h = emb.view(-1, input_dim) @ W1 + b1
h.shape

In [ ]:
W2 = torch.randn((hidden_dim, vocab_size), device=device)
b2 = torch.randn(vocab_size, device=device)

In [ ]:
logits = h @ W2 + b2
logits.shape

In [ ]:
loss = F.cross_entropy(logits, Y)
loss

In [ ]:
device = use_gpu(False)

In [ ]:
g = torch.Generator(device=device).manual_seed(seed)
X = X.to(device=device)
Y = Y.to(device=device)
C = torch.randn(vocab_size, emb_size, generator=g, requires_grad=True, device=device)
W1 = torch.randn(input_dim, hidden_dim, generator=g, requires_grad=True, device=device)
b1 = torch.randn(hidden_dim, generator=g, requires_grad=True, device=device)
W2 = torch.randn(hidden_dim, vocab_size, generator=g, requires_grad=True, device=device)
b2 = torch.randn(vocab_size, generator=g, requires_grad=True, device=device)
params = [C, W1, b1, W2, b2]
n_params = sum([p.nelement() for p in params])
n_params

In [ ]:
batch_size = 32

In [ ]:
lre = torch.linspace(-3, 0, 1000)
lrs = 10**lre

1. So low loss because of single data point of 62 examples! 3584 params for 62 examples = overfitting single batch
2. minibatch SGD
3. Cuda
4. Learning rate determiner
5. Learning rate plateau

In [ ]:
%%time

lri = []
lossi = []

for i in range(1000):
  # minibatch SGD
  idx = torch.randint(0, len(X), (batch_size, ))
  # forward pass
  emb = C[X[idx]]
  h = torch.tanh(emb.view(-1, input_dim) @ W1 + b1) 
  logits = h @ W2 + b2
  loss = F.cross_entropy(logits, Y[idx])  
  # backward pass
  for p in params:
    p.grad = None
  loss.backward()  
  
  lr = lrs[i]
  # lr = 10**-0.8
  # lr = 0.1
  for p in params:
    p.data += -lr * p.grad
    
  lri.append(lre[i])
  lossi.append(loss.item())
    
print(loss.item())    

In [ ]:
  emb = C[X]
  h = torch.tanh(emb.view(-1, input_dim) @ W1 + b1) 
  logits = h @ W2 + b2
  loss = F.cross_entropy(logits, Y)  
  print(f'{loss.item():0.4f}')

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10, 8))
ax.plot(lri, lossi)

## Dataset Splits and full training

In [ ]:
block_size = 3 # contenxt length: how many characters do we take to predict the next character
emb_size = 30
hidden_dim = 150
input_dim = block_size * emb_size
batch_size = 4096
training_loss, dev_loss = [],[]
X,Y = build_dataset(texts, stoi)

In [ ]:
device = use_gpu(True)
X_train,Y_train,X_dev,Y_dev,X_test,Y_test = split_datasets(X, Y, device=device)

In [ ]:
lossi = []
lre = torch.linspace(-3, 1, 1000)

first_run = True
g = torch.Generator(device=device).manual_seed(seed)
C = torch.randn(vocab_size, emb_size, generator=g, requires_grad=True, device=device)
W1 = torch.randn(input_dim, hidden_dim, generator=g, requires_grad=True, device=device)
b1 = torch.randn(hidden_dim, generator=g, requires_grad=True, device=device)
W2 = torch.randn(hidden_dim, vocab_size, generator=g, requires_grad=True, device=device)
b2 = torch.randn(vocab_size, generator=g, requires_grad=True, device=device)

In [ ]:
params = [C,W1,b1,W2,b2]
n_params = sum([p.nelement() for p in  params])
print(f"Number of params: {n_params}")
print(f"Training set size: {X_train.nelement()}")
print(f"Dev set size: {X_dev.nelement()}")
print(f"Test set size: {X_dev.nelement()}")

In [ ]:
%%time

if not first_run:
  X_train,Y_train,X_dev,Y_dev = X_train.to(device),Y_train.to(device),X_dev.to(device),Y_dev.to(device)
  C,W1,b1,W2,b2 = C.to(device),W1.to(device),b1.to(device),W2.to(device),b2.to(device)
  params = [C,W1,b1,W2,b2]

for i in range(1000):
  idx = torch.randint(0, len(X_train), (batch_size, ))
  
  # forward pass
  emb = C[X_train[idx]]
  h = torch.tanh(emb.view(-1, input_dim) @ W1 + b1) 
  logits = h @ W2 + b2  
  loss = F.cross_entropy(logits, Y_train[idx])
  reg_term = 0.01 * ((W1**2).mean() + (W2**2).mean())
  loss += reg_term
  
  # backward pass
  for p in params:
    p.grad = None
  loss.backward()  
  # lr = 10**lre[i]
  lr = 0.1
  for p in params:
    p.data += -lr * p.grad
    
  lossi.append(loss.item())

first_run = False

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10, 8))
ax.plot(range(len(lossi)), torch.tensor(lossi))

In [ ]:
X_train,Y_train,X_dev,Y_dev = X_train.cpu(),Y_train.cpu(),X_dev.cpu(),Y_dev.cpu()
params = [p.cpu() for p in params]

In [ ]:
training_loss.append(np.round(calc_loss(X_train, Y_train, params, batch_size), 3))
dev_loss.append(np.round(calc_loss(X_dev, Y_dev, params, batch_size), 3))
print(f'Training Loss: {training_loss[-1]}')
print(f'Dev Loss: {dev_loss[-1]}')

In [ ]:
X_test,Y_test = X_test.cpu(),Y_test.cpu()
print(f'Test Loss: {np.round(calc_loss(X_test, Y_test, params, batch_size), 3)}')

In [ ]:
sns.lineplot(data=pd.DataFrame([training_loss, dev_loss], index=['Training', 'Dev']).T)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10, 8))
ax.plot(lre, lossi)

In [ ]:
torch.save([C.cpu(), W1.cpu(), b1.cpu(), W2.cpu(), b2.cpu()], f'{model_dir}/mlp.pt')
pickle.dump(log_lossi, open(f'{model_dir}/training_loss.pl', 'wb'))

In [ ]:
params = torch.load(f'{model_dir}/mlp.pt')
log_lossi = pickle.load(open(f'{model_dir}/training_loss.pl', 'rb'))

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10, 8))
ax.plot(range(len(log_lossi)), log_lossi)

## Plot the embedding

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10, 8))
ax.scatter(C[:, 0].data, C[:, 1].data, s=200)
for i in range(C.shape[0]):
  ax.text(C[i, 0].item(), C[i, 1].item(), itos[i], ha='center', va='center', color='white')
  ax.grid('minor')

## Sample from the model

In [ ]:
C, W1, b1, W2, b2 = C.cpu(), W1.cpu(), b1.cpu(), W2.cpu(), b2.cpu()

In [ ]:
g = torch.Generator().manual_seed(seed)
n_headlines = 20
for _ in range(n_headlines):  
  ai_onion = []
  context = [0] * block_size # initialize with all ###
  while True:
    emb = C[torch.tensor([context])] # (1, block_size)
    h = torch.tanh(emb.view(1, -1) @ W1 + b1)
    logits = h @ W2 + b2
    probs = F.softmax(logits, dim=1)
    idx = torch.multinomial(probs, num_samples=1, replacement=True, generator=g).item()
    context = context[1:] + [idx]
    ai_onion.append(itos[idx])
    # if idx == 0 or len(ai_onion) > np.random.randint(50, 100):
    if idx == 0:
      break
  print(' '.join(ai_onion[:-1]))